# Epetra tutorial

## Preparation

Declare header file location. This is the installation folder of the Trilinos header files.

In [ ]:
.I /root/install/do-conf-tp-serial/include/

Load the libepetra.so library from Trilinos.

In [ ]:
.L libepetra

Now, we should have access to all Epetra functionality.

## Create communicator

Create a Comm object.

In [ ]:
#include "Epetra_SerialComm.h"

In [ ]:
Epetra_SerialComm Comm;

## Create a Map

In [ ]:
#include "Epetra_Map.h"

In [ ]:
int NumMyElements = 20;

In [ ]:
Epetra_Map Map(-1, NumMyElements, 0, Comm);

## Create a CRS Matrix

In [ ]:
#include "Epetra_CrsMatrix.h"

In [ ]:
int NumGlobalElements = Map.NumGlobalElements();

In [ ]:
Epetra_CrsMatrix A(Copy, Map, 3);

Create a tridiagonal matrix with stencil $[-1,2,-1]$.

In [ ]:
double negOne = -1.0;
double posTwo = 2.0;
for (int i=0; i<NumMyElements; i++) {
    int GlobalRow = A.GRID(i); int RowLess1 = GlobalRow - 1; int RowPlus1 = GlobalRow + 1;

    if (RowLess1!=-1) A.InsertGlobalValues(GlobalRow, 1, &negOne, &RowLess1);
    if (RowPlus1!=NumGlobalElements) A.InsertGlobalValues(GlobalRow, 1, &negOne, &RowPlus1);
    A.InsertGlobalValues(GlobalRow, 1, &posTwo, &GlobalRow);
};

A.FillComplete();

In [ ]:
std::cout << A << std::endl;

## Define Power Method

In [ ]:
#include "Epetra_Vector.h"

In [ ]:
double powerMethod (const Epetra_Operator& A, const int niters, const double tolerance)
{
  using std::cout;
  using std::endl;

  // An Operator doesn't have a Comm, but its domain Map does.
  const Epetra_Comm& comm = A.OperatorDomainMap ().Comm ();
  const int myRank = comm.MyPID ();

  // Create three vectors for iterating the power method.  Since the
  // power method computes z = A*q, q should be in the domain of A and
  // z should be in the range.  (Obviously the power method requires
  // that the domain and the range are equal, but it's a good idea to
  // get into the habit of thinking whether a particular vector
  // "belongs" in the domain or range of the matrix.)  The residual
  // vector "resid" is of course in the range of A.
  Epetra_Vector q (A.OperatorDomainMap ());
  Epetra_Vector z (A.OperatorRangeMap ());
  Epetra_Vector resid (A.OperatorRangeMap ());

  // Local error code for use below.
  int lclerr = 0;

  // Fill the iteration vector z with random numbers to start.  Don't
  // have grand expectations about the quality of our pseudorandom
  // number generator; this is usually good enough for eigensolvers.
  lclerr = z.Random ();

  // lambda: the current approximation of the eigenvalue of maximum magnitude.
  // normz: the 2-norm of the current iteration vector z.
  // residual: the 2-norm of the current residual vector "resid"
  double lambda = 0.0;
  double normz = 0.0;
  double residual = 0.0;

  const double zero = 0.0;
  const double one = 1.0;

  // How often to report progress in the power method. 
  const int reportFrequency = 10;

  // Do the power method, until the method has converged or the
  // maximum iteration count has been reached.
  for (int iter = 0; iter < niters; ++iter) {

    z.Norm2 (&normz);         // Compute the 2-norm of z
    q.Scale (one / normz, z); // q := z / normz
    A.Apply (q, z);           // z := A * q
    q.Dot (z, &lambda);       // Approx. max eigenvalue

    // Compute and report the residual norm every reportFrequency
    // iterations, or if we've reached the maximum iteration count.
    if (iter % reportFrequency == 0 || iter + 1 == niters) {

      resid.Update (one, z, -lambda, q, zero); // z := A*q - lambda*q
      resid.Norm2 (&residual); // 2-norm of the residual vector

      if (myRank == 0) {
        cout << "Iteration " << iter << ":" << endl
             << "- lambda = " << lambda << endl
             << "- ||A*q - lambda*q||_2 = " << residual << endl;
      }
    }
    if (residual < tolerance) {
      if (myRank == 0) {
        cout << "Converged after " << iter << " iterations" << endl;
      }
      break;
    } else if (iter + 1 == niters) {
      if (myRank == 0) {
        cout << "Failed to converge after " << niters << " iterations" << endl;
      }
      break;
    }
  }

  return lambda;
}

## Test power method

In [ ]:
powerMethod(A, 1500, 1e-8);